# ProxSkip

## Постановка задачи

Рассмотрим следующую задачу минимизации:
$$ \min_{x \in \mathbb{R}^d} (f(x) + \psi(x))\ \ \ \ \ \ \ \ (1)$$
где $f: \mathbb{R}^d \rightarrow \mathbb{R}$ - выпуклая функция, $\psi: \mathbb{R}^d \rightarrow \mathbb{R} \cup \{\infty\}$ - регуляризатор.

### Proximal Gradient Descent
Proximal Gradient Descent - подход к решению задачи $(1)$. Это итеративный алгоритм со следующим шагом:
$$ x_{t+1} = prox_{\gamma_t \psi}(x_t  - \gamma_t \nabla f(x_t)) $$
Где:
1. $x_t$ - приближение ответа в момент времени $t$.
2. $\gamma_t$ - шаг в момент времени $t$.
3. $prox_{\gamma \psi} := argmin_{y \in \mathbb{R}^d} \left( \frac12 \|y-x\|^2 + \gamma \psi(y) \right)$ - оператор приближения.

Как правило, вычисление градиента в Proximal Gradient Descent является более вычислительно сложным, чем вычисление оператора приближения. Однако в данной статье рассматривается ситуация, когда оператор приближения по сложности вычисления сравним с градиентом.

### Распределенные вычисления
Рассмотрим следующую задачу. Пусть есть $n$ кластеров/нод/вычислительных клиентов, и $i$-ый кластер вычисляет функцию $f_i: \mathbb{R}^d \rightarrow \mathbb{R}$. Рассмотрим задачу минимизации функции $f(x) := \frac1n \sum\limits_{i=1}^n f_i(x)$:
$$ \min_{x \in \mathbb{R}^d} f(x) $$
Эта задача актуальна для современного машинного обучения, т.к. является абстракцией над задачей минимизации эмпирического риска. \
Посмотрим на частный случай задачи $(1)$ в распределенном виде:
$$ \min_{x_1, \dots, x_n \in \mathbb{R}^d} \frac1n \sum\limits_{i=1}^{n} f_i(x_i) + \psi(x_1, \dots, x_n)$$
где $\psi(x_1, \dots, x_n) = 0$ при $x_1 = \dots = x_n$, $+\infty$ иначе. \
При такой задаче локальный подсчет функции $f_i$ на $i$-ом кластере - не очень сложная вычислительная задача, а главная трудность кроется в коммуникации между кластерами. \
Подобные задачи возникают в федеративном обучении. Разрабатываются алгоритмы для сокращения коммуникации и достижения хорошего временного соотношения для коммуникации и вычислений. Бегло посмотрим на вклад данной статьи в решении подобных задач.

### Идеи и обощения ProxSkip
ProxSkip - обобщение метода Proximal Gradient Descent для решения задачи $(1)$. Суть метода заключается в том, что вместо вычисления значения оператора $prox$ на каждом шаге, он вычисляется с некоторой вероятностью $p \in (0, 1]$. \
Для того, чтобы эффективность метода была доказуема, используется валидационное слагаемое $h_t$. \
В статье описывается Scaffnew - метод применения ProxSkip к задачам федеративного обучения. 

Так же рассматриваются следующие **расширения** этого метода:
1. Переход от детерминированного вычисления градиента к стохастическому.
2. Переход от вычислений с центральным сервером к децентрализованным вычислениям.

## Реализация в коде и эксперименты

In [2]:
import jax
import numpy as np
import jax.numpy as jnp
import scipy.stats as sps

Начнем с реализации алгоритма ProxSkip в общем виде.

In [3]:
def ProxSkip(gamma, p, x0, h0, T, f, nabla_f, phi):
    '''
    gamma - stepsize
    p - probability of skipping the prox
    x0 - initial iterate
    h0 - initial control variate
    T - number of iterations
    f - smooth function
    nabla_f - grad of smooth function
    phi - proper, closed and convex reqularizer
    '''
    curr_x = x0
    curr_h = h0
    
    coin = sps.bernoulli(p)
    
    for t in range(T):
        hat_x = curr_x - gamma * (nabla_f(curr_x) - curr_h)
        
        calc_prox = coin.rvs(size=1)[0]
        
        if calc_prox:
            prox_func = lambda x : ((gamma * phi(x)) / p)
            curr_x = prox(prox_func, hat_x - (gamma * curr_h) / p)
        else:
            curr_x = hat_x
            
        curr_h += p * (curr_x - hat_x) / gamma
        
    return curr_x   

Перейдем к реализации Scaffnew - применения ProxSkip-а к задачам федеративного обучения. Здесь предполагается, что у нас есть несколько параллельно работающих кластеров, и функция $g$ принимает массив $x$ и возвращает массив, $i$-ая компонента которого - $g_i(x_i)$, причем компоненты вычисляются параллельно и независимо друг от друга.

В методе используется функция update_x, которая должна параллельно для $i=1\dots n$ делать следующее: если $calc\_prox[i] = 1$, то заменить $curr\_x[i]$ на $\frac1n \sum\limits_{i=1}^{n} \hat x [i]$, иначе заменить $curr\_x[i]$ на $\hat x [i]$. 


In [4]:
def Scaffnew(gamma, p, x0, h0, T, g, update_x):
    '''
    gamma - stepsize
    p - probability of skipping the prox
    x0 - initial iterate, an array with equal components
    h0 - initial control variate, an array with zero sum of elements
    T - number of iterations
    g - a function to be computed in parallel
    update_x - parralel calculation of curr_x
    '''
    curr_x = x0
    curr_h = h0
    n = len(x0)
    
    coin = sps.bernoulli(p)
    
    for t in range(T):
        calc_prox = coin.rvs(size=n)
        
        hat_x = curr_x - gamma * (g(curr_x) - curr_h)
        
        update_x(curr_x, hat_x, calc_prox)
        
        curr_h += p * (curr_x - hat_x) / gamma
        
    return curr_x

Перейдем к реализации Decentralized Scaffnew - обобщения Scaffnew, где вместо обычного среднего вычисляется взвешенное среднее.

In [5]:
def DecentralizedScaffnew(gamma, tau, p, x0, h0, W, T, f, nabla_f):
    '''
    gamma, tau - stepsizes
    p - probability of scipping the prox
    x0 - initial iterate, an array with equal components
    h0 - initial control variate, an array with zero elements
    W - weights for averaging
    f - smooth function
    grad_f - gradient of f
    '''
    curr_x = x0
    curr_h = h0
    n = len(x0)
    
    coin = sps.bernoulli(p)
    
    for t in range(T):
        hat_x = curr_x - gamma * (nabla_f(curr_x) - curr_h)
        
        calc_prox = coin.rvs(size=n)
        
        for i in range(n):
            if calc_prox[i]:
                k = (gamma * tau) / p
                curr_x[i] = (1 - k) * hat_x[i] + k * np.dot(W[i], x.T)
                curr_h[i] += p * (curr_x[i] - hat_x[i]) / gamma
            else:
                curr_x[i] = hat_x[i]
                # curr_h[i] = curr_h[i] - remains the same
            
        return curr_x